## 0. Getting Setup

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

[INFO] Couldn't find torchinfo... installing it.


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## 1. Getting Data

In [4]:
def download_data(source : str,
                  destination : str,
                  remove_source: bool = True):
  """Downloads a zipped dataset from source and unzips to destination.
    Args:
        source (str): A link to a zipped file containing data.
        destination (str): A target directory to unzip data to.
        remove_source (bool): Whether to remove the source after downloading and extracting.
  """
  import os
  import pathlib
  import requests
  import zipfile

  data_path = pathlib.Path("data/")
  image_path = data_path / destination

  if image_path.is_dir():
    print(f"[INFO] {image_path} directory exists, skipping download.")
  else:
    print(f"[INFO] Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents = True, exist_ok = True)

  target_file = pathlib.Path(source).name
  with open(data_path /target_file, "wb") as f:
    request = requests.get(source)
    print(f"[INFO] Downloading {target_file} from {source}...")
    f.write(request.content)

  with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
    print(f"[INFO] Unzipping {target_file} data...")
    zip_ref.extractall(image_path)

  if remove_source:
            os.remove(data_path / target_file)

  return image_path

In [5]:
data_20_percent_path = download_data(source = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination = "pizza_steak_sushi_20_percent",
                                     )
data_20_percent_path

[INFO] Did not find data/pizza_steak_sushi_20_percent directory, creating one...
[INFO] Downloading pizza_steak_sushi_20_percent.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip...
[INFO] Unzipping pizza_steak_sushi_20_percent.zip data...


PosixPath('data/pizza_steak_sushi_20_percent')

In [6]:
train_dir = data_20_percent_path / "train"
test_dir = data_20_percent_path / "test"
train_dir, test_dir

(PosixPath('data/pizza_steak_sushi_20_percent/train'),
 PosixPath('data/pizza_steak_sushi_20_percent/test'))

## 2. FoodVision Mini model with EffNetB2

### 2.1 Creating a function to make an EffNetB2 feature extractor

In [7]:
def create_effnetb2_model(num_classes : int = 3, seed : int = 42):
  weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  transforms = weights.transforms()
  model = torchvision.models.efficientnet_b2(weights = weights)

  for params in model.parameters():
    params.requires_grad = False

  torch.manual_seed(seed)
  model.classifier = nn.Sequential(
      nn.Dropout(p = 0.3),
      nn.Linear(in_features = 1408, out_features = num_classes)
  )

  return model, transforms

In [8]:
effnetb2, effnetb2_transforms = create_effnetb2_model(num_classes=3, seed=42)

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 69.7MB/s]


In [9]:
summary(model = effnetb2,
        input_size = (1, 3, 224, 224),
        col_names = ["input_size", "output_size", "num_params", "trainable"],
        col_width = 20,
        row_settings = ["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 3]               --                   Partial
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 1

### 2.2 Creating DataLoaders for EffNetB2

In [10]:
import os
num_workers = os.cpu_count()
def create_dataLoaders(train_dir: str,
                       test_dir: str,
                       transform : torchvision.transforms.Compose,
                       batch_size: int,
                       num_workers: int = num_workers):
  train_dataset = torchvision.datasets.ImageFolder(train_dir, transform = transform)
  test_dataset = torchvision.datasets.ImageFolder(test_dir, transform = transform)

  classes = train_dataset.classes

  train_dataLoader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True, num_workers = num_workers, pin_memory = True)
  test_dataLoader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False, num_workers = num_workers, pin_memory = True)

  return train_dataLoader, test_dataLoader, classes

In [11]:
batch_size = 32
effnetb2_train_dataLoader, effnetb2_test_dataLoader, effnetb2_classes = create_dataLoaders(train_dir, test_dir, effnetb2_transforms, batch_size = batch_size)

In [13]:
effnetb2_train_dataLoader, effnetb2_test_dataLoader, effnetb2_classes

(<torch.utils.data.dataloader.DataLoader at 0x7b7a2d64a150>,
 ['pizza', 'steak', 'sushi'])

### 2.3 Training EffNetB2

In [14]:
optimizer = torch.optim.Adam(effnetb2.parameters(), lr = 0.001)
loss_fn = nn.CrossEntropyLoss()

In [15]:
from tqdm.auto import tqdm
def train_step(model: torch.nn.Module,
               dataLoader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device):
  model.train()
  train_loss, train_acc = 0, 0
  for batch, (X, y) in enumerate(dataLoader):
    X, y = X.to(device), y.to(device)
    optimizer.zero_grad()
    train_logits = model(X)
    loss = loss_fn(train_logits, y)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()

    train_preds = torch.argmax(torch.softmax(train_logits, dim = 1), dim = 1)
    train_acc += (train_preds == y).sum().item() / len(train_logits)

  train_loss = train_loss / len(dataLoader)
  train_acc = train_acc / len(dataLoader)

  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataLoader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device
              ):
  model.eval()
  test_loss, test_acc = 0, 0
  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataLoader):
      X, y = X.to(device), y.to(device)
      test_logits = model(X)
      loss = loss_fn(test_logits, y)
      test_loss += loss.item()
      test_preds = torch.argmax(test_logits, dim = 1)
      test_acc += ((test_preds == y).sum().item() / len(test_preds))
    test_loss = test_loss / len(dataLoader)
    test_acc = test_acc / len(dataLoader)

  return test_loss, test_acc

def train_test(model : torch.nn.Module,
               train_dataLoader : torch.utils.data.DataLoader,
               test_dataLoader : torch.utils.data.DataLoader,
               optimizer : torch.optim.Optimizer,
               loss_fn : torch.nn.Module,
               epochs : int,
               device : torch.device):

  results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

  model.to(device)

  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model = model,
                                       dataLoader = train_dataLoader,
                                       loss_fn = loss_fn,
                                       optimizer = optimizer,
                                       device = device
                                       )
    test_loss, test_acc = test_step(model = model,
                                    dataLoader = test_dataLoader,
                                    loss_fn = loss_fn,
                                    device = device)

    print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  return results

In [16]:
effnetb2_results = train_test(model = effnetb2,
           train_dataLoader = effnetb2_train_dataLoader,
           test_dataLoader = effnetb2_test_dataLoader,
           optimizer = optimizer,
           loss_fn = loss_fn,
           epochs = 10,
           device = device)

effnetb2_results

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9677 | train_acc: 0.5417 | test_loss: 0.7279 | test_acc: 0.9437
Epoch: 2 | train_loss: 0.7054 | train_acc: 0.8521 | test_loss: 0.5700 | test_acc: 0.9500
Epoch: 3 | train_loss: 0.5679 | train_acc: 0.8771 | test_loss: 0.4896 | test_acc: 0.9256
Epoch: 4 | train_loss: 0.5103 | train_acc: 0.8792 | test_loss: 0.4172 | test_acc: 0.9443
Epoch: 5 | train_loss: 0.4319 | train_acc: 0.8979 | test_loss: 0.3750 | test_acc: 0.9597
Epoch: 6 | train_loss: 0.4296 | train_acc: 0.8958 | test_loss: 0.3443 | test_acc: 0.9318
Epoch: 7 | train_loss: 0.3576 | train_acc: 0.9271 | test_loss: 0.3292 | test_acc: 0.9597
Epoch: 8 | train_loss: 0.3568 | train_acc: 0.8771 | test_loss: 0.3425 | test_acc: 0.9290
Epoch: 9 | train_loss: 0.3761 | train_acc: 0.8479 | test_loss: 0.2918 | test_acc: 0.9381
Epoch: 10 | train_loss: 0.3111 | train_acc: 0.9167 | test_loss: 0.2698 | test_acc: 0.9597


{'train_loss': [0.9677491227785746,
  0.7054300586382548,
  0.5679320971171061,
  0.5103442589441936,
  0.43191425999005634,
  0.429578572511673,
  0.35761584440867106,
  0.35679728388786314,
  0.3761371195316315,
  0.31106174091498057],
 'train_acc': [0.5416666666666666,
  0.8520833333333333,
  0.8770833333333333,
  0.8791666666666667,
  0.8979166666666667,
  0.8958333333333334,
  0.9270833333333334,
  0.8770833333333333,
  0.8479166666666667,
  0.9166666666666666],
 'test_loss': [0.7278638362884522,
  0.5699584007263183,
  0.48962814211845396,
  0.41724650263786317,
  0.374985808134079,
  0.34429955780506133,
  0.3292296648025513,
  0.34248256385326387,
  0.2917785316705704,
  0.2698476403951645],
 'test_acc': [0.94375,
  0.95,
  0.9255681818181818,
  0.9443181818181818,
  0.959659090909091,
  0.9318181818181819,
  0.959659090909091,
  0.9289772727272727,
  0.9380681818181819,
  0.959659090909091]}